In [19]:
import numpy as np
import pandas as pd
from qiskit_optimization import QuadraticProgram

In [20]:
# Example: select dataset
dataset_option = 'NASDAQ'  # or 'NASDAQ', 'Crypto'

# Build file paths
expected_returns_path = f'results/{dataset_option}/expected_returns.csv'
cov_matrix_path = f'results/{dataset_option}/cov_matrix.csv'

# Load data
expected_returns = pd.read_csv(expected_returns_path, index_col=0)
cov_matrix = pd.read_csv(cov_matrix_path, index_col=0)

In [21]:
# Calculate Sharpe ratio for each asset (assuming risk-free rate = 0)
asset_volatility = np.sqrt(np.diag(cov_matrix))
sharpe_ratio = expected_returns.values.flatten() / asset_volatility

In [22]:
# Get indices of top 15 assets by Sharpe ratio
N = 15
sorted_indices = np.argsort(sharpe_ratio)[::-1][:N]
top_assets = expected_returns.index[sorted_indices]

In [37]:
print(top_assets)

Index(['AppLovin Corporation', 'Palantir Technologies Inc.',
       'T-Mobile US, Inc.', 'Costco Wholesale Corporation',
       'MicroStrategy Incorporated', 'Netflix, Inc.', 'Axon Enterprise, Inc.',
       'NVIDIA Corporation', 'Broadcom Inc.', 'Intuitive Surgical, Inc.',
       'DoorDash, Inc.', 'Meta Platforms, Inc.', 'Booking Holdings Inc.',
       'CrowdStrike Holdings, Inc.', 'Apple Inc.'],
      dtype='object', name='Company')


In [23]:

# Filter expected returns and covariance matrix for top assets
mu = expected_returns.loc[top_assets].values.flatten()
Sigma = cov_matrix.loc[top_assets, top_assets].values
n_assets = N

In [24]:
# UI selection: 'low', 'medium', or 'high'
gamma_option = 'medium'  # Change as needed

# Map UI options to gamma values
gamma_map = {
    'low': 0.1,
    'medium': 0.5,
    'high': 1.0
}

gamma = gamma_map.get(gamma_option, 0.5)  # Default to 0.5 if not found
print(f"Gamma value set to: {gamma}")

Gamma value set to: 0.5


In [25]:


# Flexible budget: you can choose either by number or percentage
# Example: set budget_type = "percentage" and budget_value = 0.3 (30%)
#          or set budget_type = "number" and budget_value = 10 (10 stocks)

budget_type = "number"   # options: "percentage" or "number"
budget_value = 5         # if percentage → fraction of total assets, if number → count

if budget_type == "percentage":
    budget = int(n_assets * budget_value)
else:  # "number"
    budget = int(budget_value)

# Ensure budget is within valid range
budget = max(1, min(budget, n_assets))
print(f"Budget constraint: selecting {budget} out of {n_assets} assets")

Budget constraint: selecting 5 out of 15 assets


In [26]:
qp = QuadraticProgram("Portfolio Optimization")

In [27]:
# Decision variables: x_i = 1 if asset i is chosen, 0 otherwise
for i in range(n_assets):
    qp.binary_var(name=f"x_{i}")

In [28]:
# Objective: maximize returns - gamma * risk
# Equivalent to: minimize -mu^T x + gamma * x^T Σ x
linear = -mu
quadratic = gamma * Sigma
qp.minimize(linear=linear, quadratic=quadratic)

In [29]:
# Budget constraint: select exactly "budget" assets
qp.linear_constraint(
    linear={f"x_{i}": 1 for i in range(n_assets)},
    sense="==",
    rhs=budget,
    name="budget_constraint"
)

<LinearConstraint: x_0 + x_1 + x_10 + x_11 + x_12 + x_13 + x_14 + x_2... == 5 'budget_constraint'>

In [30]:
import numpy as np
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import QAOA
from qiskit.primitives import Sampler
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer

In [31]:
sampler = Sampler()
qaoa = QAOA(sampler=sampler, optimizer=COBYLA(), reps=1)
optimizer = MinimumEigenOptimizer(qaoa)

In [32]:
result = optimizer.solve(qp)

In [33]:
print("\nOptimal portfolio allocation:")
print(result)


Optimal portfolio allocation:
fval=-0.01965647157242505, x_0=1.0, x_1=1.0, x_2=0.0, x_3=0.0, x_4=1.0, x_5=0.0, x_6=1.0, x_7=1.0, x_8=0.0, x_9=0.0, x_10=0.0, x_11=0.0, x_12=0.0, x_13=0.0, x_14=0.0, status=SUCCESS


In [38]:
selected_assets = [top_assets[i] for i, x in enumerate(result.x) if x > 0.5]
print("\nSelected Portfolio Assets:", selected_assets)


Selected Portfolio Assets: ['AppLovin Corporation', 'Palantir Technologies Inc.', 'MicroStrategy Incorporated', 'Axon Enterprise, Inc.', 'NVIDIA Corporation']


In [41]:
# ---- Portfolio Weights based on Expected Returns ----

# Extract chosen assets (from QAOA solution)
# Assuming `result` contains the solution dictionary with chosen assets (x_i = 1 if selected)
selected_assets = [i for i, val in enumerate(result.x) if val > 0.5]

chosen_returns = expected_returns.iloc[selected_assets].values.flatten()

# Compute proportional weights
weights = chosen_returns / chosen_returns.sum()

# Total investment
total_investment = 1_000_000
allocation = weights * total_investment

# Create allocation DataFrame
portfolio_allocation = pd.DataFrame({
    "Asset": top_assets[selected_assets],
    "Expected Return": chosen_returns,
    "Weight": weights,
    "Investment (₹)": allocation
})

print("\nPortfolio Allocation (Proportional to Expected Returns):")
print(portfolio_allocation)


Portfolio Allocation (Proportional to Expected Returns):
                        Asset  Expected Return    Weight  Investment (₹)
0        AppLovin Corporation         0.000714  0.202767   202767.360582
1  Palantir Technologies Inc.        -0.000416 -0.118353  -118353.267940
2  MicroStrategy Incorporated         0.001136  0.322841   322840.922986
3       Axon Enterprise, Inc.         0.001439  0.408918   408917.953266
4          NVIDIA Corporation         0.000647  0.183827   183827.031106


In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator
from qiskit_algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer

# Authenticate and set up service (replace with your IBM Cloud API token)
service = QiskitRuntimeService(
    channel="ibm_quantum",
    token="JkDkowkioNQxX9u-JJLGOlDdYr6d9I_NJLKvFsn55a2C"
)

# Choose backend (e.g., 'ibmq_qasm_simulator' or a real device like 'ibm_manila')
backend = "ibmq_qasm_simulator"  # or e.g. "ibm_manila"

# Set up Sampler primitive for QAOA
sampler = Sampler(backend=backend, service=service)

# Set up QAOA and optimizer
qaoa = QAOA(sampler=sampler)
optimizer = MinimumEigenOptimizer(qaoa)

# Solve the problem
result = optimizer.solve(qp)

# Display results
print('Optimal selection:', result.x)
print('Objective value:', result.fval)
selected_assets = [top_assets[i] for i in range(n_assets) if result.x[i] == 1]
print('Selected assets:', selected_assets)

RequestsApiError: 'HTTPSConnectionPool(host=\'auth.quantum.ibm.com\', port=443): Max retries exceeded with url: /api/version (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x78aa3ffd0530>: Failed to resolve \'auth.quantum.ibm.com\' ([Errno -2] Name or service not known)"))'